# Team 3 - Merchant Category Recommendation
## Feature Extraction

### Team 3
- Vinicio De Sola
- Kevin Hanna
- Pri Nonis
- Bradley Nott

The results from Feature Engineering should get cleaned up here before modeling. 

In [1]:
import numpy               as np
import matplotlib.pyplot   as plt
import pandas              as pd
import seaborn             as sb

from datetime              import timedelta, datetime

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.linear_model  import LinearRegression
from sklearn.model_selection import KFold

from matplotlib.colors     import LogNorm

from IPython.display       import HTML, Markdown

from sklearn.metrics       import mean_squared_error

import lightgbm as lgb

%matplotlib inline

np.random.seed(0)

import os
print(os.listdir("./input"))

['historical_transactions.csv', '.ipynb_checkpoints', 'new_merchant_transactions.csv', 'engineered_test.pkl', '.~lock.Data_Dictionary.xlsx#', 'merchants.csv', 'sample_submission.csv', 'foo.csv', 'test.csv', 'Data_Dictionary.xlsx', 'features_importance_kfold.pkl', 'engineered_train.pkl', '.gitignore', 'train.csv']


In [5]:
train = pd.read_pickle('input/engineered_train.pkl')
test  = pd.read_pickle('input/engineered_test.pkl')

target = train['target']
del      train['target']

agg_feature_importance  = pd.read_pickle('input/features_importance_kfold.pkl')
agg_feature_importance_cols = agg_feature_importance.index

# reduce training set to top 20
top_cols = 20
train = train[agg_feature_importance_cols[:top_cols]]
test = test[agg_feature_importance_cols[:top_cols]]

display(train.head())
display(train.head())

,old_month_diff_mean,old_month_lag_std,old_month_lag_mean,first_active_delta_days,old_purchase_amount_median,new_purchase_amount_max,old_category_1_sum,old_purchase_amount_min,new_month_lag_mean,old_purchase_amount_max,old_category_1_merchants_sum,old_installments_mean,old_month_diff_std,old_authorized_flag_sum,old_category_4_merchants_sum,old_purchase_amount_mean,old_purchase_amount_std,old_merchant_id_nunique,old_purchase_amount_sum,new_purchase_amount_min
card_id,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.052830,2.402947,-3.867925,246,-0.698613,-0.296112,0,-0.739395,1.478261,2.258395,24.0,0.015094,0.224118,252,247.0,-0.639279,0.210298,94,-169.408819,-0.724368
C_ID_3d0044924f,0.958904,3.794709,-4.917808,397,-0.707839,-0.701858,35,-0.742400,1.500000,4.630299,62.0,1.616438,0.198785,354,336.0,-0.596651,0.381557,142,-217.777537,-0.739410
C_ID_d639edf6cd,-0.022727,3.832376,-8.681818,550,-0.699552,-0.700326,0,-0.730138,2.000000,-0.145847,2.0,0.000000,0.150756,42,41.0,-0.679050,0.086497,13,-29.878220,-0.700326
C_ID_186d6a6901,-0.044944,1.831235,-2.853933,154,-0.709342,-0.566740,13,-0.740897,1.714286,1.445596,30.0,1.101124,0.208355,89,87.0,-0.647467,0.247943,50,-57.624584,-0.734135
C_ID_cdbd2c0db2,-0.021127,1.022302,-1.260563,93,-0.677861,0.450886,16,-0.746156,1.555556,7.193041,34.0,1.345070,0.144316,137,134.0,-0.369525,1.309894,66,-52.472549,-0.739395


,old_month_diff_mean,old_month_lag_std,old_month_lag_mean,first_active_delta_days,old_purchase_amount_median,new_purchase_amount_max,old_category_1_sum,old_purchase_amount_min,new_month_lag_mean,old_purchase_amount_max,old_category_1_merchants_sum,old_installments_mean,old_month_diff_std,old_authorized_flag_sum,old_category_4_merchants_sum,old_purchase_amount_mean,old_purchase_amount_std,old_merchant_id_nunique,old_purchase_amount_sum,new_purchase_amount_min
card_id,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.052830,2.402947,-3.867925,246,-0.698613,-0.296112,0,-0.739395,1.478261,2.258395,24.0,0.015094,0.224118,252,247.0,-0.639279,0.210298,94,-169.408819,-0.724368
C_ID_3d0044924f,0.958904,3.794709,-4.917808,397,-0.707839,-0.701858,35,-0.742400,1.500000,4.630299,62.0,1.616438,0.198785,354,336.0,-0.596651,0.381557,142,-217.777537,-0.739410
C_ID_d639edf6cd,-0.022727,3.832376,-8.681818,550,-0.699552,-0.700326,0,-0.730138,2.000000,-0.145847,2.0,0.000000,0.150756,42,41.0,-0.679050,0.086497,13,-29.878220,-0.700326
C_ID_186d6a6901,-0.044944,1.831235,-2.853933,154,-0.709342,-0.566740,13,-0.740897,1.714286,1.445596,30.0,1.101124,0.208355,89,87.0,-0.647467,0.247943,50,-57.624584,-0.734135
C_ID_cdbd2c0db2,-0.021127,1.022302,-1.260563,93,-0.677861,0.450886,16,-0.746156,1.555556,7.193041,34.0,1.345070,0.144316,137,134.0,-0.369525,1.309894,66,-52.472549,-0.739395
